In [69]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [70]:
file = 'sample_data\\201805\\RL7785--1K-20180517102718_crv.csv'
file_rlt = 'sample_data\\201805\\RL7785--1K-20180517102718_rlt.csv'

In [134]:
data = pd.read_csv(file, sep=',', encoding='shift_jis', skiprows=1)
data_rlt = pd.read_csv(file_rlt, sep=',', encoding='shift_jis')
'''
sigma = 3*F*L / (2*b*h^2)
epsilon = 6*100*s*h / (L^2)
F: load, L:span = 64 mm, b:width, h:height, s=strain/mm
'''
L = 64
b = data_rlt.iloc[2, 3]; h = data_rlt.iloc[2, 4]
print('width, height of sample 1:', b, ',', h)

width, height of sample 1: 9.815 , 3.945


In [136]:
data = data.iloc[:,1:3]
data.columns = ['strain', 'stress']

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [82]:
data_rlt.head()

,ｻﾝﾌﾟﾙ,状,合,幅,厚さ,弾性率,最大点,破壊点,最大点.1,破壊点.1,...,破壊点.3,破壊点.4,Unnamed: 485,Unnamed: 486,Unnamed: 487,Unnamed: 488,Unnamed: 489,Unnamed: 490,Unnamed: 491,Unnamed: 492
0,No.,態,否,NaN,NaN,NaN,強度,強度,曲げ伸び,曲げ伸び,...,変位,標線変位,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,mm,mm,MPa,MPa,MPa,%,%,...,mm,mm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,NaN,9.815,3.945,10068,88.0,84.0,1.77,1.88,...,3.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,NaN,NaN,9.817,3.941,10058,89.2,89.1,1.79,1.85,...,3.206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,NaN,NaN,9.816,3.944,10037,87.8,86.6,1.74,1.83,...,3.171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


width, height of sample 1: 9.815 , 3.945


In [89]:
plt.plot(data['mm'], data['N'])
plt.xlabel = 'strain / %'
plt.ylabel = 'stress / N'

KeyError: 'mm'